In [1]:
!pip install torchvision==0.8.2

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     |████████████████████████████████| 12.8 MB 17.4 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 89.6 MB/s eta 0:00:01


In [2]:
import torch
import numpy as np
import os
import torch_neuron
from torchvision import models
import logging
from datetime import datetime as dt

## Enable logging so we can see any important warnings
logger = logging.getLogger('Neuron')
logger.setLevel(logging.INFO)

image = torch.zeros([1, 3, 224, 224], dtype=torch.float32)

## Load a pretrained ResNet50 model
model = models.resnet50(pretrained=True)

## Tell the model we are using it for evaluation (not training)
model.eval()

## Analyze the model - this will show operator support and operator count
torch.neuron.analyze_model(model, example_inputs=[image])

## Now compile the model - with logging set to "info" we will see
## what compiles for Neuron, and if there are any fallbacks
## (If using DLAMI v36 or older, please add option "compiler_args='-O2'" to the trace command)
start = dt.now()
model_neuron = torch.neuron.trace(model, example_inputs=[image])
print(f'Compilation took {(dt.now() - start).seconds} seconds')

## Export to saved model
model_neuron.save("compiled/resnet50_neuron.pt")

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/ubuntu/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
24.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

67.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100.0%
INFO:Neuron:The following operations are currently supported in torch-neuron for this model:
INFO:Neuron:prim::ListConstruct
INFO:Neuron:aten::_convolution
INFO:Neuron:aten::batch_norm
INFO:Neu

Compilation took 88 seconds


In [10]:
!ls -la compiled

total 40388
drwxrwxr-x 2 ubuntu ubuntu     4096 Jan 28 17:20 .
drwxrwxr-x 8 ubuntu ubuntu     4096 Jan 28 17:20 ..
-rw-rw-r-- 1 ubuntu ubuntu 41345679 Jan 28 17:11 resnet50_neuron.pt
